# Setup Scratch

In [ ]:
import os

# scratch directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
! [ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"

scratch_path = os.environ.get('SCRATCH', 'scratch')

In [ ]:
# fetch all env variable these values will be fetched from Data Connection setup

access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

In [ ]:
# create s3 bucket
# upload files to bucket
# download files from bucket

import boto3
import minio

In [ ]:

s3 = boto3.resource(
    "s3",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret,
    config=boto3.session.Config(signature_version="s3v4"),
    verify=False,
    endpoint_url=s3_endpoint,
)
s3_client = s3.meta.client

bucket = s3.Bucket(bucket_name)

objects = bucket.objects.filter(Prefix="/")
for obj in objects:
    path, filename = os.path.split(obj.key)
    if len(path) != 0 and not os.path.exists(os.path.join(local_dest_dir, path)):
        os.makedirs(os.path.join(local_dest_dir, path))
    if not obj.key.endswith("/"):
        download_to = local_dest_dir + path + "/" + filename if path else filename
        s3_client.download_file(bucket_name, obj.key, download_to)